In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [7]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "hsuI7rFcrWxGbUYbNQr2q5eB5"
    consumer_secret = "pG6a5HY6mCJ8kBEpZePJSYfhrVGImQEfNAkW5b6Jbl1S2xIeuo"
    access_token = "1516539997439946752-dcZaqmR9ci7q2PPb0tgpJd43iwxfhR"
    access_token_secret = "DU2kEuyvTXiHrMlFx5d5yjDse4uHsAU4jIg3b6EK4w2yY"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                  -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1518421671438340096, 'username': 'OchoBuckets', 'created_at': 'Mon Apr 25 02:48:32 +0000 2022', 'lng': -85.7223455, 'lat': 38.195555, 'text': '“My taxes shouldn’t have to take care of your kids” the kid you lobbied for me not be able to prevent and limit acc… https://t.co/NtqVaLU85d'}
{'id': 1518421674089197572, 'username': 'Bowman1965', 'created_at': 'Mon Apr 25 02:48:32 +0000 2022', 'lng': -74.7243235, 'lat': 40.073040500000005, 'text': '@JCope222 @Jillybean2374 Night Jen.'}
{'id': 1518421674315718656, 'username': 'testgeo2021', 'created_at': 'Mon Apr 25 02:48:32 +0000 2022', 'lng': -118.4119065, 'lat': 34.0207895, 'text': 'TEST_GEO_Hierarchy: d4a2f2b7-b7b9-4c65-a110-a501daf8f14c'}
{'id': 1518421674328211459, 'username': 'bardot_Barbie', 'created_at': 'Mon Apr 25 02:48:32 +0000 2022', 'lng': -73.8311875, 'lat': 40.6708795, 'text': 'Who IS DIS BOOO THANG @TRINArockstarr  USE 2 DATE ... HIM WAS CUTEEEE 🍷🍷😍😍😍😍🥰❗️❗️❗️......BUT WHEN THERE IS NO… https://t.co/MI0OR9NtIL'}
{'id': 151

KeyboardInterrupt: ignored